Tracking Objects in Video with Particle Filters
===============================================

Import libraries

In [25]:
import numpy as np
import cv2

from ipynb.fs.full.fonctions import display
# Repeatability
np.random.seed(0)

VFILENAME = "video_record/1310.mp4"
HEIGHT = 406
WIDTH = 722

Load video frames from file

In [26]:
def get_frames(filename):
    video=cv2.VideoCapture(filename)
    #print(np.shape(video))
    while video.isOpened():
        ret,frame=video.read()
        if ret:
            yield frame
        else:
            break
    video.release()
    yield None

Creating a particle cloud

In [27]:
NUM_PARTICLES=200
VEL_RANGE=0.5

def initialize_particles(N=NUM_PARTICLES,velocity=VEL_RANGE,width=WIDTH,height=HEIGHT):
    particles=np.random.rand(N,4)
    particles=particles*np.array((height,width,velocity,velocity))
    particles[:,2:4] -= velocity/2.0
    #print(particles[:20,:])
    return particles

Moving particles according to their velocity state

In [28]:
def apply_velocity(particles):
    particles[:,0] += particles[:,2]
    particles[:,1] += particles[:,3]
    return particles

Prevent particles from falling off the edge of the video frame

In [29]:
def enforce_edges(particles,N=NUM_PARTICLES,width=WIDTH,height=HEIGHT):
    for i in range(N):
        
        #print("     before: ",particles[i,0],particles[i,1],particles[i,2],particles[i,3])
        vitesse=40
        
        if particles[i,0]>height-1:
            particles[i,0]=height-1-vitesse
            particles[i,2]=-vitesse
        if particles[i,1]> width-1:
            particles[i,1]=width-1-vitesse
            particles[i,3]=-vitesse
            
        if particles[i,0]< 0:
            particles[i,0]=vitesse
            particles[i,2]=vitesse
        if particles[i,1]< 0:
            particles[i,1]=vitesse
            particles[i,3]=vitesse
            
        #print("     after: ",particles[i,0],particles[i,1])
            
    return particles

Measure each particle's quality

In [30]:
TARGET_COLOUR = np.array((156,74,38))
TARGET_COLOUR_WHITE = np.array((45,40,205))
DIFF_COLOR=(150,100,100)

def compute_errors(particles, frame,N=NUM_PARTICLES,colour=TARGET_COLOUR_WHITE, diff=DIFF_COLOR):
    color_lower=colour-diff
    errors=np.zeros(N)
    for i in range(N):
        y=int(particles[i,0])
        x=int(particles[i,1])
        
        #print("     pixel: ",x,y)
        
        pixel_colour=frame[y,x,:]

        if all(color_lower<pixel_colour):
            errors[i]=np.sum((colour-pixel_colour)**2)
            #print(color_lower,pixel_colour,errors[i])
        else:
            errors[i]=10000000
            
        #errors[i]=np.sum((colour-pixel_colour)**2)
        
    return errors

Assign weights to the particles based on their quality of match

In [31]:
def compute_weights(errors,particles,width=WIDTH,height=HEIGHT):
    weights=np.max(errors)-errors
    weights[
        (particles[:,0]==0) |
        (particles[:,0] == height-1) |
        (particles[:,1]==0) |
        (particles[:,1] == width-1) 
    ]=0.0
    weights=weights**4
    return weights

Resample particles according to their weights

In [32]:
def resample(particles, weights,N=NUM_PARTICLES):
    weights+=1.e-100
    somme=np.sum(weights)
    
    probabilities = weights/somme
    index_numbers=np.random.choice(
        N,
        size=N,
        p=probabilities
    )
    
    particles=particles[index_numbers,:]
    y=np.mean(particles[:,0])
    x=np.mean(particles[:,1])
    return particles, (int(x), int(y))

Fuzz the particles

In [33]:
POS_SIGMA = 1.0
VEL_SIGMA = 0.5
def apply_noise(particles,N=NUM_PARTICLES,sigma=POS_SIGMA):
    noise=np.concatenate(
        (
        np.random.normal(0.0, sigma, (N,1)),
        np.random.normal(0.0, sigma, (N,1)),
        np.random.normal(0.0, sigma, (N,1)),
        np.random.normal(0.0, sigma, (N,1)),
    ),
    axis=1)
    particles+=noise
    return particles

Display the video frames

In [34]:
def display_particle(frame, particles, location,N=NUM_PARTICLES):
    if len(particles) > 0:
        for i in range(N):
            y=int(particles[i,0])
            x=int(particles[i,1])
            cv2.circle(frame,(x,y),1,(0,255,0),5)
    if len(location) > 0:
        cv2.circle(frame, (location[0],location[1]), 15,(0,0,255),5)
    
    display(frame,name='particle')
    
    if cv2.waitKey(30)==27:
        if cv2.waitKey(0)==27:
            return True
            
    return False

Main routine

In [35]:
TARGET_COLOUR = np.array((156,74,38))
TARGET_COLOUR_WHITE = np.array((200,200,200))
POS_SIGMA = 1.0
VEL_SIGMA = 0.5
NUM_PARTICLES=200
VEL_RANGE=500
DIFF_COLOR=(150,100,100)

WIDTH=1920
HEIGHT=1080

def particlesDetect(particles,frame,N=NUM_PARTICLES,width=WIDTH,height=HEIGHT,colour=TARGET_COLOUR_WHITE,sigma=POS_SIGMA,diff_color=DIFF_COLOR):
    
    particles = apply_velocity(particles)
    particles = enforce_edges(particles,N=N,width=width,height=height)
    errors = compute_errors(particles, frame, N=N,colour=colour,diff=diff_color)
    weights = compute_weights(errors,particles,width=width,height=height)
    particles, location = resample(particles, weights,N=N)
    particles = apply_noise(particles,N=N,sigma=sigma)
    terminate = display_particle(frame, particles, location,N=N)
    
    if np.array_equal(frame[location[1],location[0],:],[0,0,0]):
        location=None
    
    return particles,terminate,location

In [36]:
"""
W=1920
H=1080
print(W,H)
N=10
POS_SIGMA = 10.0
VEL=50.0
particles = initialize_particles(N=N,width=W,height=H,velocity=VEL)

TARGET_COLOUR_WHITE = np.array((200,200,200))
for frame in get_frames(VFILENAME):
    #print(frame.shape)
    #print(frame[1080-1,1920-1,: ])
    if frame is None: break

    particles,terminate,location=particlesDetect(particles,frame,N=N,width=W,height=H,sigma=POS_SIGMA)
    
    if terminate:break
"""

'\nW=1920\nH=1080\nprint(W,H)\nN=10\nPOS_SIGMA = 10.0\nVEL=50.0\nparticles = initialize_particles(N=N,width=W,height=H,velocity=VEL)\n\nTARGET_COLOUR_WHITE = np.array((200,200,200))\nfor frame in get_frames(VFILENAME):\n    #print(frame.shape)\n    #print(frame[1080-1,1920-1,: ])\n    if frame is None: break\n\n    particles,terminate,location=particlesDetect(particles,frame,N=N,width=W,height=H,sigma=POS_SIGMA,coeffW_error=150)\n    \n    if terminate:break\n'